In [1]:
import os 
os.getcwd()

'/Users/liuzitian/coding/CSP/Week 1/Day 3'

In [2]:
# on first time run
#!pip install --user scikit-learn
#!pip install --user nltk

# import packages and stopwords list
import pandas as pd

import nltk
nltk.download('stopwords')

#nltk.download('punkt')

from nltk.corpus import stopwords

# use English stopwords
stops = set(stopwords.words("english"))

# read the data - replace with your own data
df = pd.read_csv("trump_insult_tweets_2014_to_2021.csv")

# drop data with missing values in the 'Review' column
df = df.dropna(axis=0, subset=['tweet'])

# convert the relevant column to lowercase
df['tweet'] = df.tweet.str.lower() 

# create a list of the reviews from the 'Reviews' column
words = df['tweet'].tolist()

# tokenise the words
from nltk.tokenize import word_tokenize  

word_tokens = []
for review in words:
    word_tokens.append(word_tokenize(review))  

# create a new list with stop words removed
filtered_reviews = []
for review in word_tokens:
    filtered_reviews.append([w for w in review if not w in stops])

# add the tokens to the dataframe    
df['tokens'] = filtered_reviews

# stem the tokens
from nltk.stem import PorterStemmer 
ps = PorterStemmer() 

stemmed = []
for review in filtered_reviews: 
    stemmed.append([ps.stem(w) for w in review])
    
# add to the dataframe
df['tokens'] = stemmed

# put the tokens back together as text
import string
rejoin = []
for review in stemmed:
    x = ",".join(review) # join the text back together 
    x = x.replace(",", " ") # replace commas with spaces
    # remove punctuation from the reviews using the string package
    rejoin.append(x.translate(str.maketrans('', '', string.punctuation)))

# add the reformed text to the data frame    
df['filtered_tweet'] = rejoin

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liuzitian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
df.head()

,Unnamed: 0,date,target,insult,tweet,tokens,filtered_tweet
0,1,2014-10-09,thomas-frieden,fool,"can you believe this fool, dr. thomas frieden ...","[believ, fool, ,, dr., thoma, frieden, cdc, ,,...",believ fool dr thoma frieden cdc state ...
1,2,2014-10-09,thomas-frieden,DOPE,"can you believe this fool, dr. thomas frieden ...","[believ, fool, ,, dr., thoma, frieden, cdc, ,,...",believ fool dr thoma frieden cdc state ...
2,3,2015-06-16,politicians,all talk and no action,big time in u.s. today - make america great ag...,"[big, time, u.s., today, -, make, america, gre...",big time us today make america great politic...
3,4,2015-06-24,ben-cardin,It's politicians like Cardin that have destroy...,politician @senatorcardin didn't like that i s...,"[politician, @, senatorcardin, n't, like, said...",politician senatorcardin nt like said baltimo...
4,5,2015-06-24,neil-young,total hypocrite,"for the nonbeliever, here is a photo of @neily...","[nonbeliev, ,, photo, @, neilyoung, offic, $, ...",nonbeliev photo neilyoung offic request—t...


In [4]:
df.to_csv('filtered_Trump.csv')

In [5]:
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
tags = pos_tag(word_tokens[0]) # POS tag the first review (before filtering)
tags

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/liuzitian/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('can', 'MD'),
 ('you', 'PRP'),
 ('believe', 'VB'),
 ('this', 'DT'),
 ('fool', 'NN'),
 (',', ','),
 ('dr.', 'NN'),
 ('thomas', 'VBD'),
 ('frieden', 'NN'),
 ('of', 'IN'),
 ('cdc', 'NN'),
 (',', ','),
 ('just', 'RB'),
 ('stated', 'VBN'),
 (',', ','),
 ('``', '``'),
 ('anyone', 'NN'),
 ('with', 'IN'),
 ('fever', 'NN'),
 ('should', 'MD'),
 ('be', 'VB'),
 ('asked', 'VBN'),
 ('if', 'IN'),
 ('they', 'PRP'),
 ('have', 'VBP'),
 ('been', 'VBN'),
 ('in', 'IN'),
 ('west', 'NN'),
 ('africa', 'NN'),
 ("''", "''"),
 ('dope', 'NN')]

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# set vectorizer - CountVectorizer for word counts and TfidfVectorizer for TF-IDF
vectorizer = CountVectorizer()
# vectorizer = TfidfVectorizer()

# create an array of word counts / TF-IDF scores
doc_vec = vectorizer.fit_transform(df.tweet)

# convert this to a dataframe
df2 = pd.DataFrame(doc_vec.toarray(), columns=vectorizer.get_feature_names())

# set a threshold to drop infrequent words
threshold = 0.1

# drop words based on the threshold
df2 = df2.drop(df2.mean()[df2.mean() < threshold].index.values, axis=1)

# join the two datasets together
df = df.join(df2, how='left')
#df = df.concat(df, df2)

In [ ]:
df.head()

In [ ]:
df.to_csv('filtered_Trump_vector.csv')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# set vectorizer - CountVectorizer for word counts and TfidfVectorizer for TF-IDF
#vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()

# create an array of word counts / TF-IDF scores
doc_vec = vectorizer.fit_transform(df.tweet)

# convert this to a dataframe
df2 = pd.DataFrame(doc_vec.toarray(), columns=vectorizer.get_feature_names())

# set a threshold to drop infrequent words
threshold = 0.1

# drop words based on the threshold
df2 = df2.drop(df2.mean()[df2.mean() < threshold].index.values, axis=1)

# join the two datasets together
df = df.join(df2, how='left')

In [ ]:
df.head()

In [ ]:
from nltk import bigrams

string = "I like Liping and Jordan but they are not as good as Michael."
bigrm = list(nltk.bigrams(string.split())) # split the sentence to words
bigrm

In [ ]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

# create a single string that combines 200 reviews
# words is the list of reviews created earlier
long_string = word_tokenize(",".join(words[0:200]))

# Bigrams
finder = BigramCollocationFinder.from_words(long_string)
# only bigrams that appear >= 2x
finder.apply_freq_filter(2)
# return the 10 bigrams with the highest likelihood
print(finder.nbest(bigram_measures.likelihood_ratio, 10))

In [ ]:
# Trigrams
finder = TrigramCollocationFinder.from_words(long_string)
# only bigrams that appear >= 2x
finder.apply_freq_filter(2)
# return the 10 trigrams with the highest likelihood
print(finder.nbest(trigram_measures.likelihood_ratio, 10))

In [ ]:
 !pip install --user textblob
from textblob import TextBlob

# calculate polarity and subjectivity
df['Polarity'] = df['tweet'].apply(lambda review: TextBlob(str(review)).sentiment.polarity)
df['Subjectivity'] = df['tweet'].apply(lambda review: TextBlob(str(review)).sentiment.subjectivity)

# sort by polarity (low to high)
sorted_df = df.sort_values(by=['Polarity'])

# print top 5 positive and negative
print("Most positive #5 reviews ")
print(sorted_df.tweet.tail())
print("\n") # print line break
print("Most negative #5 reviews ")
print(sorted_df.tweet.head())

# write the dataframe out to csv
df.to_csv("filtered_Trump_sentiment.csv")

In [ ]:
df.to_csv("filtered_Trump_sentiment.csv")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

max_words = 500 # use only the top 500 words
k = 10 # set number of topics as 10
n_top_words = 20 # print the top 20 words for each topic

# helper function to plot topics
# see Grisel et al. 
# https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f'Topic {topic_idx +1}',
                     fontdict={'fontsize': 30})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=20)
        for i in 'top right left'.split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

# vectorise the data into word counts
tf_vectorizer = CountVectorizer(max_features=max_words, stop_words='english')
tf = tf_vectorizer.fit_transform(df['tweet'])

# fit LDA - we'll cover online learning later in the module
lda = LDA(n_components=k, max_iter=5, learning_method='online')
lda.fit(tf)

# get the list of words (feature names)
tf_feature_names = tf_vectorizer.get_feature_names()

# print the top words per topic
plot_top_words(lda, tf_feature_names, n_top_words, 'Topics in LDA model')